# Simulating MDV at Steady State

Metabolic flux analysis involves simulating the labeling pattern of metabolites at a defined flux distribution in a given metabolic network. In this tutorial, we will demonstrate how to simulate the labeling pattern of metabolites using FreeFlux. We will use a [toy model](https://github.com/Chaowu88/freeflux/tree/main/models/toy) to illustrate the process.

We start by loading the metabolic model from a tab-separated values (tsv) [file](https://github.com/Chaowu88/freeflux/blob/main/models/toy/reactions.tsv). We then create a Simulator object for steady-state simulation ('ss'). We call the `set_target_EMUs method` to specify the EMUs whose MDVs need to be simulated.

In [1]:
from freeflux import Model

MODEL_FILE = 'path/to/reactions.tsv'

model = Model('demo')
model.read_from_file(MODEL_FILE)

sim = model.simulator('ss')
sim.set_target_EMUs({
    'Glu': [[1,2,3], '12345'],  # EMU "Glu_123" and "Glu_12345"
    'AKG': [2,3],               # EMU "AKG_23"
    'Cit': '12345'              # EMU "Cit_12345"
})           

Next, we specify the labeling strategy for the metabolic network. In this toy model, the network uptakes 25% (mol%) C2 labeled and 25% fully labeled acetyl-CoA with both 100% purities.

In [2]:
sim.set_labeling_strategy(
    'AcCoA', 
    labeling_pattern = ['01', '11'], 
    percentage = [0.25, 0.25], 
    purity = [1, 1]
)

<div class="alert alert-info">

<b>Note:</b> <br></br> Call this method for each substrate if multiple substrates are used. 

</div>

We can input the defined flux distribution using the `set_flux` method or read it from a .tsv or .xslx [file](https://github.com/Chaowu88/freeflux/blob/main/models/toy/fluxes.tsv) using the `set_fluxes_from_file` method.

In [3]:
fluxes = {'v1': 10,
          'v2': 10,
          'v3': 5,
          'v4': 5,
          'v5': 5,
          'v6_f': 12.5,   # "_f" denotes forward flux
          'v6_b': 7.5,    # "_b" denotes backward flux
          'v7': 5}
for fluxid, value in fluxes.items():
    sim.set_flux(fluxid, value)
    
# or read from file
FLUXES_FILE = 'path/to/fluxes.tsv'
sim.set_fluxes_from_file(FLUXES_FILE)

We are now ready to simulate the labeling patterns:

In [4]:
sim.prepare()
res = sim.simulate()
print(res)

simulated MDVs
Glu_123: MDV([0.671, 0.21, 0.102, 0.017])
Glu_12345: MDV([0.328, 0.276, 0.274, 0.088, 0.03, 0.004])
AKG_23: MDV([0.693, 0.262, 0.044])
Cit_12345: MDV([0.328, 0.276, 0.274, 0.088, 0.03, 0.004])


The `with` statement can also be used to perform the simulation.

In [5]:
with model.simulator('ss') as sim:
    sim.set_target_EMUs({
        'Glu': [[1,2,3], '12345'], 
        'AKG': [2,3], 
        'Cit': '12345'
    })
    sim.set_labeling_strategy(
        'AcCoA', 
        labeling_pattern = ['01', '11'], 
        percentage = [0.25, 0.25], 
        purity = [1, 1]
    )
    sim.set_fluxes_from_file(FLUXES_FILE)
    sim.prepare()
    res = sim.simulate()
print(res)

simulated MDVs
Glu_123: MDV([0.671, 0.21, 0.102, 0.017])
Glu_12345: MDV([0.328, 0.276, 0.274, 0.088, 0.03, 0.004])
AKG_23: MDV([0.693, 0.262, 0.044])
Cit_12345: MDV([0.328, 0.276, 0.274, 0.088, 0.03, 0.004])


For a more complex example of steady state simulation using an *E. coli* model, you can refer to the script "[tutorial_synechocystis_inst_simulation.py](https://github.com/Chaowu88/freeflux/tree/main/tutorials)"